In [1]:
import os
import logging
from os.path import abspath
from pathlib import Path
from dataclasses import field
from typing import Optional
from datasets import load_dataset
import transformers
import torch
import numpy as np
from transformers import (
    T5Tokenizer,
    AutoTokenizer,
    T5ForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)
from wasabi import msg
from pathlib import Path
from os.path import abspath

/home/lgrootde/Generative-re-tests/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting file paths

In [2]:
home_dir = Path(abspath("")).parent
msg.info(f"Home directory: {home_dir}")

# Files used:
config_path = home_dir.joinpath('config/config_testing.yaml')

ℹ Home directory: /home/lgrootde/Generative-re-tests


# Load Config

In [2]:
# Load the config
import yaml
with open() as f:
    config = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
# Pretty print the config using json as a shortcut
import json
print(json.dumps(config, sort_keys=False, indent=4))

{
    "model_name": "google/t5-v1_1-base",
    "dataset_vars": {
        "type": "csv",
        "dir": "data/cdr_seq2rel",
        "split": "train",
        "column_names": [
            "input",
            "relations"
        ]
    },
    "output_dir": "./fine_tune_results",
    "local_rank": -1,
    "per_device_train_batch_size": 4,
    "per_device_eval_batch_size": 4,
    "gradient_accumulation_steps": 4,
    "learning_rate": 0.0002,
    "max_grad_norm": 0.3,
    "weight_decay": 0.001,
    "max_seq_length": 512,
    "max_target_length": 512,
    "num_train_epochs": 1,
    "padding": true,
    "ignore_pad_token_for_loss": true,
    "truncation": true,
    "predict_with_generate": true,
    "gradient_checkpointing": true,
    "optim": "paged_adamw_32bit",
    "lr_scheduler_type": "constant",
    "max_steps": 32,
    "warmup_ratio": 0.03,
    "group_by_length": true,
    "save_steps": 8,
    "do_eval": true,
    "evaluation_strategy": "steps",
    "eval_steps": 8,
    "pytorch_cuda_al

# Load dataset

First we'll load the dataset. The dataset used is the [CDR dataset in seq2rel format](https://github.com/JohnGiorgi/seq2rel-ds), This dataset is slightly altered to make it easier to use with [huggingface dataset](https://huggingface.co/docs/datasets/index).

In [4]:
dataset = load_dataset(
        config['dataset_vars']['type'], 
        data_dir=home_dir.joinpath(config['dataset_vars']['dir']),
        column_names=config['dataset_vars']['column_names']
        )

dataset_train = dataset['train'].select(range(1,501)) # remove first row that contains column names
dataset_eval = dataset['validation'].select(range(1,501)) # remove first row that contains column names

In [5]:
# Useful function to see a random row in the dataset
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [6]:
show_random_elements(dataset_train, 1)

,input,relations
0,"Risks of the consumption of beverages containing quinine. Although the United States Food and Drug Administration banned its use for nocturnal leg cramps due to lack of safety and efficacy, quinine is widely available in beverages including tonic water and bitter lemon. Numerous anecdotal reports suggest that products containing quinine may produce neurological complications, including confusion, altered mental status, seizures, and coma, particularly in older women. Psychologists need to inquire about consumption of quinine-containing beverages as part of an evaluation process.",quinine @CHEMICAL@ confusion @DISEASE@ @CID@ quinine @CHEMICAL@ seizures @DISEASE@ @CID@ quinine @CHEMICAL@ coma @DISEASE@ @CID@


The dataset has 2 columns, "Input" which are abstracts from various papers from pubmed and "relations" which are the relations between chemicals and diseases. The relations are in an novel format explained in the [seq2rel paper](https://aclanthology.org/2022.bionlp-1.2/). 

# Load tokenizer and model

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Apr 11 16:43:35 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100 80GB PCIe          Off | 00000001:00:00.0 Off |                    0 |
| N/A   37C    P0              61W / 300W |      4MiB / 81920MiB |     23%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [8]:
model_name = config['model_name']
device_map = {"": 0}

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, legacy=False)
model = T5ForConditionalGeneration.from_pretrained(
    model_name,
    device_map=device_map
) # we specificly use T5 for COnfitional generations because it has a language modeling head

# Changing the vocab

In [9]:
special_characters = "@CHEMICAL@ @DISEASE@ @CID@ ;"
# Tokenize the target sequence
labels = tokenizer(
    text_target=special_characters, 
    max_length=config['max_seq_length'], 
    padding=config['padding'], 
    truncation=config['truncation'],  
    return_tensors='pt'
)
labels
tokenizer.tokenize(special_characters)

['▁@',
 'CHE',
 'M',
 'ICAL',
 '@',
 '▁@',
 'D',
 'ISE',
 'ASE',
 '@',
 '▁@',
 'C',
 'ID',
 '@',
 '▁',
 ';']

# Test tokenizer and model

In [10]:
# Show a single example of the dataset
single_example = dataset_train[0]
print(f"A single row in the dataset is of type {type(single_example)}")
print(single_example)

A single row in the dataset is of type <class 'dict'>
{'input': 'Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same recepto

Before going into the model the data has to be tokenized. Tokenization is a preprocess where the text is split up into tokens, these tokens are then conferted to their respective token ids using the vocab, token ids can then be converted into word vectors which are the raw input of the model. The attention mask shows which tokens should be given attention to in the attention layer in the model. A 1 means the token is given attention to in the attention layer a 0 means the token is ignored in that process. This is important when using padding for example, you don't want the padding tokens to be factored in during the attention process. 

In [11]:
column_name_input, column_name_output = config['dataset_vars']['column_names']

In [12]:
# Show what the output of the tokenizer looks like for the input task
tokenizer_output_text = tokenizer(single_example[column_name_input])
tokenizer_output_text

{'input_ids': [1823, 24938, 782, 7211, 7, 8, 1181, 13397, 49, 324, 7, 757, 1504, 13, 3, 3903, 29, 30095, 5, 86, 73, 152, 222, 88, 17, 1601, 6, 23496, 120, 6676, 324, 7, 757, 20063, 8, 6313, 16, 1717, 1666, 11, 842, 1080, 2546, 57, 6344, 162, 10529, 3, 3903, 29, 30095, 6, 305, 12, 460, 2179, 5096, 7, 87, 8711, 6, 47, 19921, 15, 26, 42, 7211, 26, 57, 3, 29, 138, 32, 9431, 6, 3, 18189, 12, 204, 5453, 87, 8711, 5, 37, 10950, 324, 7, 757, 1504, 13, 910, 5453, 87, 8711, 491, 6977, 18, 22758, 26, 32, 102, 9, 47, 92, 14610, 7211, 26, 57, 3, 29, 9, 24938, 782, 5, 1823, 24938, 782, 2238, 410, 59, 2603, 893, 1717, 1666, 42, 842, 1080, 5, 86, 2241, 13304, 7, 45, 23496, 120, 6676, 324, 7, 757, 20063, 3, 3903, 29, 30095, 6, 335, 599, 18, 13520, 12, 335, 599, 18, 9120, 283, 6, 410, 59, 2860, 16687, 7, 15, 3437, 757, 11293, 13, 784, 519, 566, 908, 18, 29, 9, 24938, 782, 13642, 3, 29, 329, 201, 11, 3, 29, 9, 24938, 782, 6, 335, 599, 18, 13520, 12, 335, 599, 18, 7256, 283, 6, 410, 59, 2860, 3, 3903, 29,

In [13]:
# Show what the output of the tokenizer looks like for the output text
with tokenizer.as_target_tokenizer():
    tokenizer_output_rel = tokenizer(single_example[column_name_output])
tokenizer_output_rel

/home/lgrootde/Generative-re-tests/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3892: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': [491, 6977, 18, 22758, 26, 32, 102, 9, 3320, 13717, 329, 23936, 1741, 10950, 324, 7, 757, 3320, 308, 19056, 17892, 1741, 3320, 254, 4309, 1741, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

Why tokenize the output? During training the model tasks is given a certain input match the expected output. To match it it needs to know where it's going wrong, so for the calculation of loss it needs the input ids of the tokenized output text. The exact training algorithm is presuambly [teacher forcing](https://machinelearningmastery.com/teacher-forcing-for-recurrent-neural-networks/). (I say presuamly here because i don't really know what training algoritm the huggingface api uses, where it's documentated, defined in the code and if it differs between models. And teacher forcing was used for training the T5 model)

The T5 model has a max sequence input of 512. If an input exceed that it is truncated. This could cause the model to output the wrong relations because it is missing some information, certainly when a lot of important information output the conclusion of a paper is usally at the end of an abstract. So we'll check how many rows exceed the max sequence length:

In [14]:
tokenized_dataset = tokenizer(dataset_train[column_name_input])
count = 0
for idx, input_ids in enumerate(tokenized_dataset['input_ids']):
    if len(input_ids) > 512:
        # print(idx, len(input_ids))
        count+=1
print(f'Percentage that comes above max sequence length: {count/(idx+1)*100}')

Token indices sequence length is longer than the specified maximum sequence length for this model (619 > 512). Running this sequence through the model will result in indexing errors


Percentage that comes above max sequence length: 13.4


# Create Preprocessing function

In [15]:
ignore_pad_token_for_loss=config['ignore_pad_token_for_loss']

In [16]:
def preprocess_function(examples):
    '''
    This function takes a dataset of input and target sequences.
    meant to be used with the dataset.map() function
    '''

    column_name_input, column_name_output = config['dataset_vars']['column_names']

    # Split input and target
    inputs, targets = [], []
    for i in range(len(examples[column_name_input])):
        if examples[column_name_input][i] and examples[column_name_output][i]: # remove pairs where one is None
            inputs.append(examples[column_name_input][i])
            targets.append(examples[column_name_output][i])

    # Tokenize the input
    model_inputs = tokenizer(
        inputs, 
        max_length=config['max_seq_length'], 
        padding=config['padding'], 
        truncation=config['truncation'], 
        return_tensors='pt'
    )

    # Tokenize the target sequence
    labels = tokenizer(
        text_target=targets, 
        max_length=config['max_seq_length'], 
        padding=config['padding'], 
        truncation=config['truncation'],  
        return_tensors='pt'
    )

    # Replace pad tokens with -100 so they don't contribute too the loss
    if ignore_pad_token_for_loss:
        labels["input_ids"] = [
                    [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
                ]

    # Add tokenized target text to output
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

In [17]:
train_dataset = dataset_train.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on train dataset"
)

eval_dataset = dataset_eval.map(
    preprocess_function,
    batched=True,
    desc="Running tokenizer on tr|ain dataset"
)

Running tokenizer on train dataset: 100%|██████████| 500/500 [00:02<00:00, 247.83 examples/s]
Running tokenizer on tr|ain dataset: 100%|██████████| 500/500 [00:01<00:00, 298.15 examples/s]


In [18]:
# A look into the pre-tokenized dataset
single_example = train_dataset[0]
print(f"A single row in the dataset is of type {type(single_example)}")
print(single_example)

A single row in the dataset is of type <class 'dict'>
{'input': 'Naloxone reverses the antihypertensive effect of clonidine. In unanesthetized, spontaneously hypertensive rats the decrease in blood pressure and heart rate produced by intravenous clonidine, 5 to 20 micrograms/kg, was inhibited or reversed by nalozone, 0.2 to 2 mg/kg. The hypotensive effect of 100 mg/kg alpha-methyldopa was also partially reversed by naloxone. Naloxone alone did not affect either blood pressure or heart rate. In brain membranes from spontaneously hypertensive rats clonidine, 10(-8) to 10(-5) M, did not influence stereoselective binding of [3H]-naloxone (8 nM), and naloxone, 10(-8) to 10(-4) M, did not influence clonidine-suppressible binding of [3H]-dihydroergocryptine (1 nM). These findings indicate that in spontaneously hypertensive rats the effects of central alpha-adrenoceptor stimulation involve activation of opiate receptors. As naloxone and clonidine do not appear to interact with the same recepto

# Create evaluation strategy

In [19]:
#model.generate(single_example)

In [20]:
import evaluate

In [21]:
metric = evaluate.load("rouge")

In [22]:
single_eval_example=[dataset_eval[0]['relations']]
print(single_eval_example)
# whats the output of metric.compute?
metric.compute(
    predictions=single_eval_example,
    references=single_eval_example,
    rouge_types=['rouge1', 'rouge2']
)


['lithium carbonate @CHEMICAL@ neurologic depression @DISEASE@ @CID@ lithium carbonate @CHEMICAL@ cyanosis @DISEASE@ @CID@ lithium carbonate @CHEMICAL@ cardiac arrhythmia @DISEASE@ @CID@']


{'rouge1': 1.0, 'rouge2': 1.0}

In [23]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
metric.compute(predictions=predictions, references=references, rouge_types=['rouge1', 'rouge2'])

{'rouge1': 1.0, 'rouge2': 1.0}

We need to define two functions:
- a postprocess function that does the post processing of a prediction
- A compute_metrics function that takes as input an prediction and output the mean score of that prediction?

the compute metrics functions alwas takes a tuple of 

In [24]:
def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        # preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
        # labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

        return preds, labels

In [25]:
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
        
    # Replace -100s used for padding as we can't decode them
    preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=False)
    result = {k: round(v * 100, 4) for k, v in result.items()} # rounds all metric values to 4 numvers behind the comma and make them percentages
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens) # mean length of the generated sequences
    return result

# Setting up trainer

In [26]:
training_arguments = Seq2SeqTrainingArguments(
        output_dir=config['output_dir'],
        per_device_train_batch_size=config['per_device_train_batch_size'],
        gradient_accumulation_steps=config['gradient_accumulation_steps'],
        optim=config['optim'],
        save_steps=config['save_steps'],
        logging_steps=config['logging_steps'],
        learning_rate=config['learning_rate'],
        fp16=config['fp16'],
        bf16=config['bf16'],
        max_grad_norm=config['max_grad_norm'],
        max_steps=config['max_steps'],
        warmup_ratio=config['warmup_ratio'],
        group_by_length=config['group_by_length'],
        lr_scheduler_type=config['lr_scheduler_type'],
        predict_with_generate=True,
        save_total_limit=2,
        save_strategy='steps',
        load_best_model_at_end=True,
        do_eval=config['do_eval'],
        evaluation_strategy=config['evaluation_strategy'],
        eval_steps=config['eval_steps']
    )

In [27]:
data_collator = DataCollatorForSeq2Seq(
        tokenizer,
        model=model,
        label_pad_token_id=-100,
        pad_to_multiple_of=8 if config['fp16'] else None,
    )

In [28]:
trainer = Seq2SeqTrainer(
        model=model,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        args=training_arguments,
    )

# Training

In [29]:
trainer.train()

Step,Training Loss,Validation Loss


KeyboardInterrupt: 

# For compute metrics function

In [30]:
predictions = trainer.predict(eval_dataset)

/home/lgrootde/Generative-re-tests/venv/lib/python3.12/site-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [33]:
predictions.predictions.shape

(500, 20)

In [34]:
predictions.label_ids.shape

(500, 403)

In [36]:
predictions.label_ids[:10].shape

(10, 403)

In [35]:
type(predictions.predictions)

numpy.ndarray

In [39]:
labels = predictions.label_ids[:10]
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
decoded_labels

['lithium carbonate @CHEMICAL@ neurologic depression @DISEASE@ @CID@ lithium carbonate @CHEMICAL@ cyanosis @DISEASE@ @CID@ lithium carbonate @CHEMICAL@ cardiac arrhythmia @DISEASE@ @CID@',
 'phenobarbital @CHEMICAL@ dyskinesia @DISEASE@ @CID@',
 'ammonia ; nh3 @CHEMICAL@ drowsiness @DISEASE@ @CID@ valproic acid ; vpa @CHEMICAL@ drowsiness @DISEASE@ @CID@',
 'haloperidol @CHEMICAL@ catalepsy @DISEASE@ @CID@ apomorphine @CHEMICAL@ hyperactivity @DISEASE@ @CID@',
 'isoproterenol ; iso @CHEMICAL@ cardiac hypertrophy @DISEASE@ @CID@ isoproterenol ; iso @CHEMICAL@ hyperplasia @DISEASE@ @CID@',
 'butylated hydroxyanisole ; bha @CHEMICAL@ forestomach carcinogenesis ; forestomach tumorigenesis ; forestomach tumors @DISEASE@ @CID@',
 'alfentanil @CHEMICAL@ muscle rigidity ; rigidity @DISEASE@ @CID@',
 'edrophonium @CHEMICAL@ bradycardias @DISEASE@ @CID@',
 'fentanyl @CHEMICAL@ muscular rigidity @DISEASE@ @CID@',
 'epsilon-aminocaproic acid @CHEMICAL@ sagittal sinus thrombosis @DISEASE@ @CID@']